In [1]:
import faiss
import pickle

# Load FAISS index
index = faiss.read_index("faiss_index.idx")

# Load text chunks
with open("retrieved_text_chunks.pkl", "rb") as file:
    text_chunks = pickle.load(file)

print(" FAISS index and text chunks loaded successfully.")


 FAISS index and text chunks loaded successfully.


### Load Embedding Model
To convert queries into embeddings, we reload our sentence-transformers model.

In [2]:
from sentence_transformers import SentenceTransformer

# Load embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")


E:\RAG_Chatbot\rag_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Retrieve Relevant Chunks
Since we already implemented retrieval in the FAISS notebook, we will reuse it.

In [3]:
def retrieve_top_k(query, k=3):
    """Retrieve top-k relevant text chunks based on query"""
    
    # Convert query to embedding
    query_embedding = embedder.encode([query])
    
    # Search FAISS index
    distances, indices = index.search(query_embedding, k)
    
    # Retrieve text chunks
    retrieved_texts = [text_chunks[i] for i in indices[0]]
    
    return retrieved_texts


### Generate an Answer
We will now pass the retrieved text to an LLM. 

Using a Local LLM (Hugging Face)

In [5]:
from transformers import pipeline

# Load a small local model (distilGPT-2 for efficiency)
generator = pipeline("text-generation", model="distilgpt2")

def generate_response_local(query):
    """Generate response using retrieved context & local model"""
    retrieved_texts = retrieve_top_k(query)
    context = " ".join(retrieved_texts)

    prompt = f"Context: {context}\n\nUser Query: {query}\nAnswer:"
    
    response = generator(prompt, max_new_tokens=50, num_return_sequences=1)
    
    return response[0]["generated_text"]

# Test Local Chatbot
query = "How Artificial Intelligence is Reshaping the Workforce?"
answer = generate_response_local(query)

print(f" Local Chatbot Answer:\n{answer}")


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Local Chatbot Answer:
Context: How Artificial Intelligence is Reshaping the Workforce: The Future of Jobs Introduction Artificial Intelligence AI is no longer a distant conceptit is actively transforming industries, businesses, and the global workforce. While some fear AI will replace jobs, others believe it will create new opportunities and enhance human productivity. So, what does the future of work look like in an AIdriven world? The Impact of AI on Jobs AI is automating repetitive tasks, optimizing workflows, and also enhances human productivity and creates new opportunities. The future workforce will not be AI versus humans but AI alongside humans, working together to drive innovation and efficiency. The key to staying relevant in this evolving landscape is embracing AI, upskilling, and leveraging human creativitytraits that no machine can fully replicate. While AI can process data, it lacks emotional intelligence, creativity, and critical thinking. Strengthening these humancentr